In [59]:
from PIL import Image
from PIL import ImageFilter
import os
import numpy as np
from math import sqrt
import argparse
import cv2

INPUT_DIR = "input"
OUTPUT_DIR = "output"

def in_file(filename):
    return os.path.join(INPUT_DIR, filename)
def out_file(filename):
    return os.path.join(OUTPUT_DIR, filename)

def show_vertical(img1, img2):
    img = Image.fromarray(np.vstack((np.array(img1),np.array(img2))))
    img.show()
    return img

def show_horizontal(img1, img2):
    img = Image.fromarray(np.hstack((np.array(img1),np.array(img2))))
    img.show()
    return img



In [ ]:
image = Image.open("balao.jpeg")

#Mostra a imagem
image.show()

#Mostra o valor do pixel
print(image.getpixel((500,500)))

In [3]:


#retorna o endereco relativo da pasta
def in_path(filename):
    return os.path.join(INPUT_DIR, filename)

image = Image.open(in_path("EU.jpg"))
print(image.getpixel((500,500)))


(194, 193, 188)


In [ ]:
def triangulo(size):
    branco = (255,255,255)
    preto = (0,0,0)
    image = Image.new("RGB",(size, size), branco)
    
    for x in range(size):
        for y in range(size):
            if x < y:
                image.putpixel((x,y),preto)
    return image

triangulo(700)



In [18]:
def bandeira_franca(altura):
    #razao = 3/2
    largura = 3 * altura // 2
    barras = largura // 3
    azul = (0,85,164)
    branco = (255,255,255)
    vermelho = (239,65,53)

    image = Image.new("RGB",(largura,altura), branco)
    for x in range(barras):
        for y in range(altura):
            image.putpixel((x,y),azul)
            image.putpixel((x + 2 * barras, y), vermelho)
    
    image.show()

bandeira_franca(700)

In [46]:
def bandeirajapao(altura):
    largura = 3 * altura // 2
    branco = (255,255,255)
    vermelho = (173,35,51)
    cinza = (51,51,51)
    raio = (3 * altura // 10 )
    centro = (largura // 2, altura // 2) 

    image = Image.new("RGB",(largura,altura), branco)
    for x in range(centro[0]-raio,centro[0]+raio):

        for y in range(centro[1]-raio,centro[1]+raio):

            if (x-centro[0])**2 + (y-centro[1])**2 <= raio**2:
                image.putpixel(((x,y)),vermelho) 
    image.show()
    

bandeirajapao(700)

In [78]:
def escala_cinza(img_color):
    width, heigth = img_color.size
    img = Image.new("RGB",(width, heigth))

    for x in range(width):
        for y in range(heigth):
            pxl = img_color.getpixel((x, y))
            lum = (pxl[0]+pxl[1]+pxl[2])//3
            img.putpixel((x,y),(lum,lum,lum))
    return img

baloes = Image.open(in_file("balao.jpeg"))
pb_balao = escala_cinza(baloes)
pb_balao.save(out_file("pb_balao.jpeg"))
pb_balao.show()




In [83]:
# width largura, heigth altura
def escala_cinza(img_color):
    width, heigth = img_color.size
    img = Image.new("RGB",(width, heigth))

    for x in range(width):
        for y in range(heigth):
            pxl = img_color.getpixel((x, y))
            lum = int(0.3 * pxl[0] + 0.59 * pxl[1]+ 0.11 * pxl[2])
            img.putpixel((x,y),(lum,lum,lum))
    return img

baloes = Image.open(in_file("balao.jpeg"))
baloes.show()
pb_balao = escala_cinza(baloes)
pb_balao.save(out_file("pb_balao.jpeg"))
pb_balao.show()


In [24]:
#Filtro convolucional

def show_box_blur(filename, r=1):
    original = Image.open(os.path.join(INPUT_DIR,filename))
    filtrado = original.filter(ImageFilter.BoxBlur(r))

    show_vertical(original,filtrado)
    filtrado.save(os.path.join(OUTPUT_DIR,'{}_boxblur_{}.jpeg'.format(filename[:filename.index('.')],r)))

show_box_blur('balao.jpeg',7)


In [60]:
# Detectando Arestas

def show_vertical_edges(filename, offset=1):
    '''Abre o original e convert para escala de cinza'''
    original = Image.open(os.path.join(INPUT_DIR,filename)).convert('L')
    filtrado = original.filter(ImageFilter.Kernel(
        (3,3),
        [-1,0,1,
         -2,0,2,
         -1,0,1],
         1,
         offset))

    show_vertical(original,filtrado)
    filtrado.save(os.path.join(OUTPUT_DIR,'{}_vsobel_{}.jpeg'.format(filename[:filename.index('.')],offset)))


def show_edges(filename, direction='x',offset=0):
    '''Abre o original e convert para escala de cinza'''
    original = Image.open(os.path.join(INPUT_DIR,filename)).convert('L')
    XSOBEL = ImageFilter.Kernel(
        (3,3),
        [-1,0,1,
         -2,0,2,
         -1,0,1],
         1,
         offset)
    YSOBEL = ImageFilter.Kernel(
        (3,3),
        [-1,-2,1,
         0,0,0,
         -1,-2,1],
         1,
         offset)
    if direction == 'x':
        filtrado = original.filter(XSOBEL)
    elif direction == 'y':
        filtrado = original.filter(YSOBEL)
    else:
        vsobel = original.filter(XSOBEL)
        hsobel = original.filter(YSOBEL)
        w, h = original.size
        filtrado = Image.new('L',(w,h))

        for i in range(w):
            for j in range(h):
                value = sqrt(
                    vsobel.getpixel((i,j))**2 + hsobel.getpixel((i,j))**2)
                value = int(min(value, 255))
                filtrado.putpixel((i,j), value)


    show_vertical(original,filtrado)
    filtrado.save(os.path.join(OUTPUT_DIR,'{}_{}vsobel_{}.jpeg'.format(filename[:filename.index('.')],direction,offset)))


show_edges('estrada.jpg','t',0)

In [64]:
# chroma key
#retorna o endereco relativo da pasta
def in_path(filename):
    return os.path.join(INPUT_DIR, filename)

original = Image.open(in_path("balao.jpeg")).convert()
w, h = original.size
modificada = Image.new('L',(w,h))

for i in range(w):
    for j in range(h):
        if original.getpixel((i,j))[0] <= 120:
            valor = 0
        else:
            valor = 255
        modificada.putpixel((i,j),valor)


#original.show()
#modificada.show()
#show_vertical(original,modificada)
modificada.save(os.path.join(OUTPUT_DIR,'balao_binaria.jpeg'))



1920 1080
